In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!!pip install open3d

['Collecting open3d',
 '  Downloading open3d-0.18.0-cp310-cp310-manylinux_2_27_x86_64.whl (399.7 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/399.7 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.1/399.7 MB\x1b \x1b3.6 MB/s\x1b eta \x1b0:01:53\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.5/399.7 MB\x1b \x1b7.3 MB/s\x1b eta \x1b0:00:55\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.9/399.7 MB\x1b \x1b8.3 MB/s\x1b eta \x1b0:00:48\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.4/399.7 MB\x1b \x1b9.7 MB/s\x1b eta \x1b0:00:42\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.9/399.7 MB\x1b \x1b10.6 MB/s\x1b eta \x1b0:00:38\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.5/399.7 MB\x1b \x1b11.4 MB/s\x1b eta \x1b0:00:35\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
addict                           2.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
bea

In [4]:
!python3 --version

Python 3.10.12


In [5]:
import os
path = '/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

calculate.py		      data_list_ptlk_L_90.csv	       sensor_tpip_45_3.pcd
checkpoint		      dataset.py		       sensor_tpip_90_1.pcd
data_list_fast_key_0.csv      Global_optimizer_fast.py	       TNUTEJN016_half2.pcd
data_list_fast_key_90.csv     Global_optimizer.py	       TNUTEJN016_half_unnoise.pcd
data_list_fast_key_L_180.csv  learning3d		       TNUTEJN016.pcd
data_list_fast_key_L_90.csv   network.py		       Tokuichi_divide.py
data_list_key_0.csv	      pretrained		       T-pipe_matching.py
data_list_key_90.csv	      __pycache__		       T-pipe_test_jurai_fast.py
data_list_key_L_180.csv       Registration_test_jurai_fast.py  T-pipe_test_jurai_ptlk.py
data_list_key_L_90.csv	      Registration_test_jurai_ptlk.py  Tpipe_test_jurai_ptlk_time.ipynb
data_list_proposed_0.csv      Registration_test_jurai.py       T-pipe_test_jurai.py
data_list_proposed_90.csv     Registration_test.py	       Tpipe_test_jurai_time.ipynb
data_list_proposed_L_180.csv  S3DIS.py			       T-pipe_test.py
data_list_propos

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import h5py
import subprocess
import shlex
import json
import glob
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import minkowski
from scipy.spatial import cKDTree
from scipy.spatial.transform import Rotation
from torch import sin, cos
import open3d as o3d
from tqdm import tqdm
import torchvision
import logging
import random
import os
import numpy.linalg as LA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import copy
import time
import Global_optimizer
import open3d as o3d
import gc

In [8]:
############################################
# 位置合わせ描画
############################################
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

############################################
# 点群のダウンサンプリングと特徴量計算
############################################
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    if theta == 0:
      radius_normal = voxel_size * 2.0   # 0度
    elif theta == 90:
      radius_normal = voxel_size * 2.0   # 90度
    elif theta == "L_90":
      radius_normal = voxel_size * 2.0   # L90度
    elif theta == "L_180":
      radius_normal = voxel_size * 2.0   # L180度
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    #radius_feature = voxel_size * 5.0   # default
    if theta == 0:
      #radius_feature = voxel_size * 7.0   # 0度
      radius_feature = voxel_size * 4.0   # 0度
    elif theta == 90:
      radius_feature = voxel_size * 5.0   # 90度
    elif theta == "L_90":
      radius_feature = voxel_size * 5.0   # L90度
    elif theta == "L_180":
      #radius_feature = voxel_size * 5.0   # L180度
      radius_feature = voxel_size * 7.0   # L180度

    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size, target, source):
    #draw_registration_result(source, target, np.identity(4))
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

############################################
# RANSACによる概略マッチング
############################################
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    if theta == 0:
      distance_threshold = voxel_size * 0.015   # 0度
    elif theta == 90:
      distance_threshold = voxel_size * 0.02   # 90度
    elif theta == "L_90":
      distance_threshold = voxel_size * 0.06    # L90度
    elif theta == "L_180":
      distance_threshold = voxel_size * 0.01    # L180度
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3,
         [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    if theta == 0:
      #distance_threshold = voxel_size * 0.015   # 0度
      distance_threshold = voxel_size * 0.5   # 0度
      decrease_mu = True
      division_factor = 0.2 #1.4
      iteration_number = 100 #64
      tuple_scale = 0.3 #95
    elif theta == 90:
      #distance_threshold = voxel_size * 0.02   # 90度
      distance_threshold = voxel_size * 0.02   # 90度
      decrease_mu = True
      division_factor = 0.2 #1.4
      iteration_number = 100 #64
      tuple_scale = 0.3 #95
    elif theta == "L_90":
      #distance_threshold = voxel_size * 0.06    # L90度
      distance_threshold = voxel_size * 0.06    # L90度
      decrease_mu = True
      division_factor = 0.2 #1.4
      iteration_number = 100 #64
      tuple_scale = 0.3 #95
    elif theta == "L_180":
      #distance_threshold = voxel_size * 0.01    # L180度
      distance_threshold = voxel_size * 0.05    # L180度
      decrease_mu = True
      ivision_factor = 0.2 #1.4
      iteration_number = 100 #64
      tuple_scale = 0.3 #95
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [9]:
def rotation_matrix_to_euler_angles(R):
    # Extract angles using trigonometric relations
    roll = np.arctan2(-R[1, 2], R[2, 2])
    pitch = np.arctan2(-(R[0, 2]*np.cos(roll)), R[2, 2])
    yaw = np.arctan2(-R[0, 1], R[0, 0])
    #yaw = np.arctan2(-1, -1)

    return np.array([roll, pitch, yaw])

def coordinate_3d(axes, range_x, range_y, range_z, grid = True):
    axes.set_xlabel("x", fontsize = 14)
    axes.set_ylabel("y", fontsize = 14)
    axes.set_zlabel("z", fontsize = 14)
    axes.set_xlim(range_x[0], range_x[1])
    axes.set_ylim(range_y[0], range_y[1])
    axes.set_zlim(range_z[0], range_z[1])
    if grid == True:
        axes.grid()

def visual_vector_3d(axes, loc, vector, color = "red"):
    axes.quiver(loc[0], loc[1], loc[2],
              vector[0], vector[1], vector[2],
              color = color, lw=3)

###################
#重心計算
###################
def calculate_centroid(pointcloud):
    # Calculate the centroid (mean) of the point cloud along each axis
    centroid = np.mean(pointcloud, axis=0)
    return centroid

def translate_to_origin(pointcloud):
    # Calculate the centroid of the point cloud
    centroid = calculate_centroid(pointcloud)

    # Translate the point cloud to move the centroid to the origin
    translated_pointcloud = pointcloud - centroid

    return translated_pointcloud

###################
#Registration
###################
# ICP registration module.
class ICP:
  def __init__(self, threshold=0.1, max_iteration=100):
    self.threshold = threshold
    self.criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iteration)

  def preprocess(self, template, source):
    if self.is_tensor: template, source = template.detach().cpu().numpy(), source.detach().cpu().numpy()	# Convert to ndarray if tensors.

    if len(template.shape) > 2:
      template, source = template[0], source[0]

    # Find mean of template & source.
    self.template_mean = np.mean(template, axis=0, keepdims=True)
    self.source_mean = np.mean(source, axis=0, keepdims=True)

		# Convert to open3d point clouds.
    template_ = o3d.geometry.PointCloud()
    source_ = o3d.geometry.PointCloud()

		# Subtract respective mean from each point cloud.
    template_.points = o3d.utility.Vector3dVector(template - self.template_mean)
    source_.points = o3d.utility.Vector3dVector(source - self.source_mean)
    return template_, source_

  # Postprocess on transformation matrix.
  def postprocess(self, res):
    est_R = res.transformation[0:3, 0:3]		# ICP's rotation matrix (source -> template)
    t_ = np.array(res.transformation[0:3, 3]).reshape(1, -1)			# ICP's translation vector (source -> template)
    est_T = np.array(res.transformation)								# ICP's transformation matrix (source -> template)
    est_t = np.matmul(est_R, -self.source_mean.T).T + t_ + self.template_mean[0] 	# update predicted translation according to eq. 3
    est_T[0:3, 3] = est_t
    return est_R, est_t, est_T

  # Convert result to pytorch tensors.
  @staticmethod
  def convert2tensor(result):
    if torch.cuda.is_available(): device = 'cuda'
    else: device = 'cpu'
    result['est_R']=torch.tensor(result['est_R']).to(device).float().view(-1, 3, 3) 		# Rotation matrix [B, 3, 3] (source -> template)
    result['est_t']=torch.tensor(result['est_t']).to(device).float().view(-1, 1, 3)			# Translation vector [B, 1, 3] (source -> template)
    result['est_T']=torch.tensor(result['est_T']).to(device).float().view(-1, 4, 4)			# Transformation matrix [B, 4, 4] (source -> template)
    return result

  # icp registration.
  def __call__(self, template, source, start):
    self.is_tensor = torch.is_tensor(template)

    # tensorからo3d
    template, source = self.preprocess(template, source)

    template_vis = template
    source_vis = source
    """
    if theta == 0:
      voxel_size = 0.0122     # 0度
    elif theta == 45:
      voxel_size = 0.011     # 90度
    elif theta == 90:
      voxel_size = 0.012     # 90度
    elif theta == 135:
      voxel_size = 0.015     # 90度
    elif theta == "L_90":
      voxel_size = 0.011    # 90度
    elif theta == "L_180":
      voxel_size = 0.016   # 90度
    """
    #fast param
    if theta == 0:
      voxel_size = 0.01     # 0度
    elif theta == 90:
      voxel_size = 0.011     # 90度
    elif theta == "L_90":
      voxel_size = 0.01    # 90度
    elif theta == "L_180":
      voxel_size = 0.012   # 90度

		### 前処理 ###
    source, template, source_down, template_down, source_fpfh, template_fpfh = Global_optimizer.prepare_dataset(voxel_size, template, source)

		### RANSACアルゴリズム（概略マッチング） ###
    #result_ransac = execute_global_registration(source_down, template_down, source_fpfh, template_fpfh, voxel_size)
    result_ransac = execute_fast_global_registration(source_down, template_down, source_fpfh, template_fpfh, voxel_size)
    elapsed_time_1 = time.time() - start

		### ICPアルゴリズム（精密マッチング） ###
    res = o3d.pipelines.registration.registration_icp(source, template, self.threshold, result_ransac.transformation, criteria=self.criteria)	# icp registration in open3d.
    elapsed_time_2 = time.time() - start

    est_R, est_t, est_T = self.postprocess(res)
    result = {'est_R': est_R,
		          'est_t': est_t,
		          'est_T': est_T}

    if self.is_tensor: result = self.convert2tensor(result)
    return result, elapsed_time_1, elapsed_time_2


# Define Registration Algorithm
def registration_algorithm():
  reg_algorithm = ICP()
  return reg_algorithm


# Register template and source pairs.
class Registration:
	def __init__(self):
		#self.reg_algorithm = reg_algorithm
		#self.is_rpmnet = True if self.reg_algorithm == 'rpmnet' else False
		#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
		self.reg_algorithm = registration_algorithm()

	@staticmethod
	def pc2points(data):
		if len(data.shape) == 3:
			return data[:, :, :3]
		elif len(data.shape) == 2:
			return data[:, :3]

	def register(self, template, source, start):
		result = self.reg_algorithm(template, source, start)
		return result

def pc2open3d(data):
	if torch.is_tensor(data): data = data.detach().cpu().numpy()
	if len(data.shape) == 2:
		pc = o3d.geometry.PointCloud()
		pc.points = o3d.utility.Vector3dVector(data)
		return pc
	else:
		print("Error in the shape of data given to Open3D!, Shape is ", data.shape)

def display_results_sample(template, source, est_T, masked_template):
  transformed_source = np.matmul(est_T[0:3, 0:3], source.T).T + est_T[0:3, 3]     # ※matmul：行列の積　　第一項：回転、第二項：平行移動、重心移動分も含まれる
  print(est_T)

  numpy_source_t = source + est_T[0:3, 3]
  source_t = o3d.geometry.PointCloud()
  source_t.points = o3d.utility.Vector3dVector(numpy_source_t)
  source_t.paint_uniform_color([0, 1, 0])

  ### 原点を表示 ###
  numpy_o = np.array([[0, 0, 0]])
  o = o3d.geometry.PointCloud()
  o.points = o3d.utility.Vector3dVector(numpy_o)
  o.paint_uniform_color([1/3, 1/3, 1/3])

  ### x軸の表示 ###
  numpy_ax_x = np.array([[0.01, 0, 0], [0.02, 0, 0], [0.03, 0, 0], [0.04, 0, 0], [0.05, 0, 0], [0.06, 0, 0], [0.07, 0, 0], [0.08, 0, 0], [0.09, 0, 0], [0.1, 0, 0]])
  ax_x = o3d.geometry.PointCloud()
  ax_x.points = o3d.utility.Vector3dVector(numpy_ax_x)
  ax_x.paint_uniform_color([1/3, 1/3, 1/3])

  ### y軸の表示 ###
  numpy_ax_y = np.array([[0, 0.01, 0], [0, 0.02, 0], [0, 0.03, 0], [0, 0.04, 0], [0, 0.05, 0], [0, 0.06, 0], [0, 0.07, 0], [0, 0.08, 0], [0, 0.09, 0], [0, 0.1, 0]])
  ax_y = o3d.geometry.PointCloud()
  ax_y.points = o3d.utility.Vector3dVector(numpy_ax_y)
  ax_y.paint_uniform_color([1/3, 1/3, 1/3])

  ### z軸の表示 ###
  numpy_ax_z = np.array([[0, 0, 0.01], [0, 0, 0.02], [0, 0, 0.03], [0, 0, 0.04], [0, 0, 0.05], [0, 0, 0.06], [0, 0, 0.07], [0, 0, 0.08], [0, 0, 0.09], [0, 0, 0.1]])
  ax_z = o3d.geometry.PointCloud()
  ax_z.points = o3d.utility.Vector3dVector(numpy_ax_z)
  ax_z.paint_uniform_color([1/3, 1/3, 1/3])

  ### 正解を定義 ###
  if theta == 0:
  	## 0度 ##
    ans_theta_x = np.radians(0)
    ans_theta_y = np.radians(1)
    ans_theta_z = np.radians(176)
  elif theta == 45:
  	## 90度 ##
    ans_theta_x = np.radians(2)
    ans_theta_y = np.radians(0.2)
    ans_theta_z = np.radians(45)
  elif theta == 90:
  	## 90度 ##
    ans_theta_x = np.radians(2)
    ans_theta_y = np.radians(0.2)
    ans_theta_z = np.radians(-90)
  elif theta == 135:
  	## 90度 ##
    ans_theta_x = np.radians(2)
    ans_theta_y = np.radians(0.2)
    ans_theta_z = np.radians(-45)
  elif theta == "L_90":
  	## 90度 ##
    ans_theta_x = np.radians(-3)
    ans_theta_y = np.radians(-1)
    ans_theta_z = np.radians(-85)
  elif theta == "L_180":
  	## 90度 ##
    ans_theta_x = np.radians(-12)
    ans_theta_y = np.radians(-2)
    ans_theta_z = np.radians(176)
  # x軸方向に回転
  R_x = np.array(
         [[1, 0, 0],
         [0, np.cos(ans_theta_x), -np.sin(ans_theta_x)],
         [0, np.sin(ans_theta_x), np.cos(ans_theta_x)]])
  # y軸方向に回転
  R_y = np.array(
         [[np.cos(ans_theta_y), 0, np.sin(ans_theta_y)],
         [0, 1, 0],
         [-np.sin(ans_theta_y), 0, np.cos(ans_theta_y)]])
  # z軸方向に回転
  R_z = np.array(
         [[np.cos(ans_theta_z), -np.sin(ans_theta_z), 0],
         [np.sin(ans_theta_z), np.cos(ans_theta_z), 0],
         [0, 0, 1]])
  # 回転行列を計算
  ans_R = R_z @ R_y @ R_x
  # 平行移動
  ans_t_ = [0, 0.008, -0.011]
  if theta == "L_90":
    ans_t_ = [0.0055, 0.008, -0.011]
  if theta == "L_180":
    ans_t_ = [0.006, 0.008, -0.011]

  # 重心移動も含めた変換を行う
  ans_t = np.matmul(ans_R, -np.mean(source, axis=0).T).T + ans_t_
  numpy_ans_source = np.matmul(ans_R, source.T).T + ans_t
  ans_source = o3d.geometry.PointCloud()
  ans_source.points = o3d.utility.Vector3dVector(numpy_ans_source)

  ### 回転移動の差分 ###
  euler_angles = rotation_matrix_to_euler_angles(est_T[0:3, 0:3])
  rotation_angle_x = np.degrees(euler_angles[0])
  rotation_angle_y = np.degrees(euler_angles[1])
  rotation_angle_z = np.degrees(euler_angles[2])
  #print("\nRotation angle around x-axis:", rotation_angle_x, "degrees")
  #print("Rotation angle around y-axis:", rotation_angle_y, "degrees")
  #print("Rotation angle around z-axis:", rotation_angle_z, "degrees")
  ###print("\n回転移動の差：")
  diff_R_x = rotation_angle_x - np.degrees(ans_theta_x)
  diff_R_y = rotation_angle_y - np.degrees(ans_theta_y)
  diff_R_z = rotation_angle_z - np.degrees(ans_theta_z)
  ###print("x軸方向　", abs(diff_R_x))
  ###print("y軸方向　", abs(diff_R_y))
  ###print("z軸方向　", abs(diff_R_z))
  global diff_R
  diff_R = np.linalg.norm([diff_R_x, diff_R_y, diff_R_z])
  print("\n回転移動の差（L2ノルム）：", diff_R)

  ### 平行移動の差分 ###
  global diff_t

  print("est_T[0:3, 3]")
  print(est_T[0:3, 3])
  print("ans_t ")
  print(ans_t)
  diff_t = np.linalg.norm(est_T[0:3, 3] - ans_t)
  print("平行移動の差（L2ノルム）：", diff_t, "\n")

  template = pc2open3d(template)
  source = pc2open3d(source)
  transformed_source = pc2open3d(transformed_source)

  template.paint_uniform_color([1, 0, 0])
  source.paint_uniform_color([0, 1, 0])
  transformed_source.paint_uniform_color([0, 1, 0])
  ans_source.paint_uniform_color([1/3, 1/3, 1/3])

  #o3d.visualization.draw_geometries([template])                                    # テンプレ
  ###o3d.visualization.draw_geometries([template, source, ans_source, o])                 # テンプレ、ソース、正解ソース、原点
  #o3d.visualization.draw_geometries([template, source, source_t, o, ax_x, ax_y, ax_z])
  #o3d.visualization.draw_plotly([template, source, transformed_source])         # テンプレ、ソース、変換後ソース、原点
  #o3d.visualization.draw_geometries([template, masked_template, source])           # テンプレ、マスクテンプレ、ソース
  #o3d.visualization.draw_geometries([template, source])                            # テンプレ、ソース
  #o3d.visualization.draw_geometries([masked_template, source])                     # マスクテンプレ、ソース

In [26]:
theta = "L_90"

# テンプレートデータのロード
#pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/TNUTEJN016.pcd"
pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/TNUTEJN016_half_unnoise.pcd"
# ソースデータ# 不要なオブジェクトを解放し、メモリ最適化する。
if theta == 0:   ## 0度 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_cheese_noise.pcd"
elif theta == 45:   ## 45度 ##
	pass
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_45_3.pcd"
elif theta == 90:   ## 90度 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_90_1.pcd"
elif theta == 135:   ## 135度 ##
	pass
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_135_2.pcd"
elif theta == "L_90":   ## L90 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_Ljoint_90_4.pcd"
	pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/TNUTEJN016_half_unnoise.pcd"
elif theta == "L_180":   ## L180 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_Ljoint_180_2.pcd"
	pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/TNUTEJN016_half_unnoise.pcd"

# テンプレとソースを点群データに変換
pcd_cheese = o3d.io.read_point_cloud(pcd_file)
pcd_cheese_rot = o3d.io.read_point_cloud(pcd_rot_file)

numpy_cheese_points = np.array(pcd_cheese.points)
numpy_cheese_rot_points = np.array(pcd_cheese_rot.points)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model_load.to(device)

n = 1000
list_time1 = []
list_time2 = []
list_timeF = []
for i in range(n):
  #####計測開始#####
  #torch.cuda.synchronize()
  # 不要なオブジェクトを解放し、メモリ最適化する。
  gc.collect()
  start = time.time()

  template_cheese = torch.tensor(numpy_cheese_points, dtype=torch.float32).unsqueeze(0)
  source_cheese = torch.tensor(numpy_cheese_rot_points, dtype=torch.float32).unsqueeze(0)
  template_cheese = template_cheese.to(device)
  source_cheese = source_cheese.to(device)

  registration_model = Registration()

	# 提案手法（MaskNet、SVD、ICP）の実行（実際のデータを代入）
  result_cheese = registration_model.register(template_cheese, source_cheese, start)
  est_T_cheese = result_cheese[0]['est_T']     # est_T：ICPの変換行列

  #####計測終了#####
  elapsed_time_F = time.time() - start
  list_time1.append(result_cheese[1])
  list_time2.append(result_cheese[2])
  list_timeF.append(elapsed_time_F)

time_mean1 = sum(list_time1) / len(list_time1)
time_mean2 = sum(list_time2) / len(list_time2)
time_meanF = sum(list_timeF) / len(list_timeF)


ストリーミング出力は最後の 5000 行に切り捨てられました。
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search

In [27]:
time_mean1

0.012150116443634033

In [28]:
time_mean2

0.0228595130443573

In [31]:
time_meanF

0.023297908544540405

In [30]:
import csv
def save_list_to_csv(filename, data):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Data'])
        for item in data:
            writer.writerow([item])
# リストの中身をCSVファイルに保存
save_list_to_csv(f'data_list_fast_key_{theta}.csv', list_timeF)